# Description
This notebook extract useful metrics from the plaques dataset or some part of
it. It relies on trackmate results to extract this information. Three metrics
are considered: number of infected cells, radius of the plaque, and radial
velocity of infected cells. Only considering infected cells, because these are
the only cells visible in the microscopy images at hand. Metrics are modeled
using a mean and standard deviation for each point in time that there is image
for it. Time is in hours post infection (hpi). Results are saved into a csv file
to be used as a reference to evaluate simulations in infectio.

# Implementation

## Part 1: choose set of files

In [1]:
import os
import pandas as pd
import numpy as np

### For WR virus    

Note: **Quick Fix**

The dataset needs to be changed to be used here. Because in many of the
of the experiments, two or more initial spots are infected and therefore center
and radii computations are not correct. As a quick fix for now, I am considering
only a few of the experiements that consist only of one plaque. For M061-WR
these are: 6, 8, 9, 11, 13, 14.

In [14]:
dataset_name = 'M061_WR_handpicked'
CSV_ROOT = "../dataset/plaques-ashkan/trackmate_output/dVGF_dF11_viruses/M061"
# include only files in range of 1 to 15 in their names, these are basic WR files
# Only consider quick fix files
single_plaque_files = [6, 8, 9, 11, 13, 14]
csv_files = [f for f in os.listdir(CSV_ROOT) if f.endswith(".csv") and int(f.split("-")[0]) in single_plaque_files]
csv_files

['8-spots.csv',
 '11-spots.csv',
 '9-spots.csv',
 '13-spots.csv',
 '6-spots.csv',
 '14-spots.csv']

### For dVGF

In [23]:
dataset_name = 'M061_dVGF_handpicked'
CSV_ROOT = "../dataset/plaques-ashkan/trackmate_output/dVGF_dF11_viruses/M061"
# include only files in range of 16 to 30 in their names, these are dVGF files
# Only consider quick fix files
single_plaque_files = [16, 17, 18, 21, 22, 23, 25, 27, 28, 29, 30]
csv_files = [f for f in os.listdir(CSV_ROOT) if f.endswith(".csv") and int(f.split("-")[0]) in single_plaque_files]
csv_files

['22-spots.csv',
 '17-spots.csv',
 '29-spots.csv',
 '30-spots.csv',
 '23-spots.csv',
 '16-spots.csv',
 '28-spots.csv',
 '25-spots.csv',
 '18-spots.csv',
 '21-spots.csv',
 '27-spots.csv']

### For dF11

In [32]:
dataset_name = 'M061_dF11_handpicked'
CSV_ROOT = "../dataset/plaques-ashkan/trackmate_output/dVGF_dF11_viruses/M061"
# include only files in range of 31 to 45 in their names, these are dF11 files
# Only consider quick fix files
single_plaque_files = [31, 32, 33, 34, 36, 37, 38, 40, 41, 44, 45]  # Remove 35 because first frames no spots
csv_files = [f for f in os.listdir(CSV_ROOT) if f.endswith(".csv") and int(f.split("-")[0]) in single_plaque_files]
csv_files

['41-spots.csv',
 '36-spots.csv',
 '40-spots.csv',
 '37-spots.csv',
 '31-spots.csv',
 '34-spots.csv',
 '45-spots.csv',
 '32-spots.csv',
 '38-spots.csv',
 '44-spots.csv',
 '33-spots.csv']

### For dVGF/dF11

In [2]:
import os
import pandas as pd
import numpy as np

dataset_name = 'M061_dVGFdF11_handpicked'
CSV_ROOT = "../dataset/plaques-ashkan/trackmate_output/dVGF_dF11_viruses/M061"
# include only files in range of 46 to 60 in their names, these are dVGF/dF11 files
# Only consider quick fix files
single_plaque_files = [46, 48, 49, 50, 51, 52, 53, 55, 57, 58, 60]  # exclude 54, 56, 59, also 47 because first few frames not enough (less than 3) spots
csv_files = [f for f in os.listdir(CSV_ROOT) if f.endswith(".csv") and int(f.split("-")[0]) in single_plaque_files]
csv_files

['55-spots.csv',
 '60-spots.csv',
 '58-spots.csv',
 '53-spots.csv',
 '46-spots.csv',
 '52-spots.csv',
 '57-spots.csv',
 '51-spots.csv',
 '48-spots.csv',
 '50-spots.csv',
 '49-spots.csv']

## Part 2: add the time stamps of the time series data

In [3]:
# Because the imaging of the dataset starts with 20 h.p.i and ends 48 hpi with
# 10 minute intervals
time_stamps = [round(x, 2) for x in np.linspace(20.0, 48.0, 169).tolist()]
refdf = pd.DataFrame({'t': time_stamps})

print(refdf)

         t
0    20.00
1    20.17
2    20.33
3    20.50
4    20.67
..     ...
164  47.33
165  47.50
166  47.67
167  47.83
168  48.00

[169 rows x 1 columns]


## Part 3: infected count metrics

In [5]:
unique_track_id_counts = []

for file in csv_files:
    df = pd.read_csv(os.path.join(CSV_ROOT, file), skiprows=[1, 2, 3], low_memory=False)
    unique_counts = df.groupby('FRAME')['TRACK_ID'].nunique()
    unique_track_id_counts.append(unique_counts)

all_counts_df = pd.concat(unique_track_id_counts, axis=1)

[[17, 6, 3, 19, 20, 19, 10, 13, 4, 9, 29], [19, 6, 3, 21, 20, 19, 10, 13, 4, 9, 30], [21, 6, 3, 20, 21, 18, 12, 13, 4, 9, 33], [21, 7, 4, 23, 20, 20, 13, 13, 3, 9, 34], [24, 9, 6, 24, 20, 20, 13, 14, 3, 11, 33], [24, 10, 7, 26, 20, 20, 13, 17, 4, 11, 34], [25, 9, 7, 25, 20, 22, 15, 17, 4, 11, 37], [26, 10, 8, 25, 20, 23, 15, 17, 5, 12, 37], [27, 10, 9, 25, 20, 22, 17, 19, 5, 13, 37], [26, 12, 10, 29, 20, 25, 17, 21, 5, 14, 37], [30, 13, 10, 29, 21, 25, 18, 21, 5, 14, 38], [32, 13, 10, 28, 21, 25, 19, 21, 5, 15, 40], [35, 13, 10, 30, 21, 25, 19, 23, 5, 16, 41], [34, 13, 10, 30, 21, 27, 19, 23, 5, 15, 42], [36, 13, 10, 30, 22, 28, 20, 25, 5, 17, 45], [37, 13, 10, 30, 24, 28, 22, 25, 5, 18, 45], [38, 13, 10, 32, 24, 28, 22, 25, 6, 18, 47], [37, 14, 10, 33, 27, 28, 24, 25, 6, 18, 49], [40, 16, 10, 33, 28, 29, 24, 27, 6, 17, 51], [42, 16, 13, 38, 29, 29, 23, 27, 7, 17, 51], [45, 16, 13, 39, 31, 30, 23, 27, 8, 17, 55], [46, 16, 12, 41, 31, 32, 23, 28, 8, 19, 57], [48, 19, 12, 42, 32, 32, 23,

In [7]:
# Adding count values to refdf
# Calculate average and standard deviation for each frame
average_counts = all_counts_df.mean(axis=1)
std_dev_counts = all_counts_df.std(axis=1)
# also add list of all numbers for 2sample test. for backward compability, keep
# the mean and std cols as well

# print(average_counts, std_dev_counts)
refdf['inf-count-mean'] = average_counts
refdf['inf-count-std'] = std_dev_counts

refdf['inf-count-list'] = all_counts_df.values.tolist()
refdf

,t,inf-count-mean,inf-count-std,inf-count-list
0,20.00,13.545455,8.029491,"[17, 6, 3, 19, 20, 19, 10, 13, 4, 9, 29]"
1,20.17,14.000000,8.473488,"[19, 6, 3, 21, 20, 19, 10, 13, 4, 9, 30]"
2,20.33,14.545455,9.070431,"[21, 6, 3, 20, 21, 18, 12, 13, 4, 9, 33]"
3,20.50,15.181818,9.400193,"[21, 7, 4, 23, 20, 20, 13, 13, 3, 9, 34]"
4,20.67,16.090909,8.971673,"[24, 9, 6, 24, 20, 20, 13, 14, 3, 11, 33]"
...,...,...,...,...
164,47.33,519.545455,190.898593,"[694, 385, 313, 742, 547, 594, 318, 416, 399, ..."
165,47.50,525.000000,194.767554,"[712, 388, 316, 752, 550, 597, 317, 418, 408, ..."
166,47.67,530.909091,194.594684,"[723, 393, 323, 764, 556, 599, 321, 426, 415, ..."
167,47.83,537.000000,198.061102,"[729, 392, 326, 775, 570, 611, 322, 427, 416, ..."


## Part 4: Area & radius reference metrics
Area would be a better metric instead of radius.

In [36]:
from scipy.spatial import ConvexHull

def get_convex_radius(points):
    if len(points) < 3:
        return 0
    hull = ConvexHull(points)
    boundary_points = points[hull.vertices]
    center = np.mean(boundary_points, axis=0)
    radii = (boundary_points - center)
    radii = np.linalg.norm(radii, axis=1)
    return radii.mean()

def get_area(points):
    if len(points) < 3:
        return 0
    hull = ConvexHull(points)
    return hull.volume  # Remember that .volume method gives the area in 2d and not .area

In [37]:
all_radii_stats = []
all_area_stats = []

for file in csv_files:
    df = pd.read_csv(os.path.join(CSV_ROOT, file), skiprows=[1, 2, 3], low_memory=False)
    points_vs_frame = df.groupby('FRAME').apply(lambda x: x[['POSITION_X', 'POSITION_Y']].values)
    radii_vs_frame = [get_convex_radius(points) for points in points_vs_frame]
    area_vs_frame = [get_area(points) for points in points_vs_frame]
    # remove any zeros in there
    radii_vs_frame = [r for r in radii_vs_frame if r != 0]
    all_radii_stats.append(radii_vs_frame)
    all_area_stats.append(area_vs_frame)

In [38]:
array_of_lists = np.array(all_radii_stats) # (Correction: apparently Fiji already counts in the pixel length and width) * 3.1746  # because this dataset, both pixel width and height are this number in microns. 
mean_radii = array_of_lists.mean(axis=0)
std_radii = array_of_lists.std(axis=0)

refdf['radius-mean(um)'] = mean_radii
refdf['radius-std(um)'] = std_radii

area_array_of_lists = np.array(all_area_stats)
mean_area = area_array_of_lists.mean(axis=0)
std_area = area_array_of_lists.std(axis=0)

refdf['area-mean(um2)'] = mean_area
refdf['area-std(um2)'] = std_area

refdf

,t,inf-count-mean,inf-count-std,radius-mean(um),radius-std(um),area-mean(um2),area-std(um2)
0,20.00,8.454545,4.390071,50.775458,16.695897,6329.897458,5058.395235
1,20.17,8.818182,4.190899,51.469313,16.068189,6738.964437,4956.375937
2,20.33,9.363636,4.433345,52.236234,16.578712,6949.618190,5200.132492
3,20.50,10.272727,4.692354,55.131957,16.693659,7757.873227,5601.432696
4,20.67,10.636364,4.822297,57.225456,15.660132,8319.521243,5540.309176
...,...,...,...,...,...,...,...
164,47.33,540.545455,103.941679,502.870874,114.122022,818956.247324,256073.083318
165,47.50,546.545455,106.215219,493.045570,105.914216,827120.347954,253250.178784
166,47.67,551.454545,105.298019,500.333248,114.702915,835214.300623,250179.149789
167,47.83,557.818182,103.831419,502.874965,114.966366,836713.306692,246823.017673


## Part 5: Radial velocity reference metrics

In [39]:
# 1. for each file:
# 2. compute the center of for all infected points in the first frame and consider it as center
# 3. for each trackid, compute maximum radial velocity
# 4. average all of these values to get the radial velocity for the file

def max_upto_each_index(arr):
    result = []
    max_index = []
    for i in range(len(arr)):
        max_index.append(np.argmax(arr[:i+1]))
        result.append(arr[max_index[-1]])
    return np.array(result), np.array(max_index)

def compute_radial_velocity_of_trackmate_csvfile(csv_file):
    # Radial velocity is computed as the average of maximum radial component (rho)
    # of tracks divided by the time spent to reach that maximum
    df = pd.read_csv(csv_file, skiprows=[1, 2, 3], low_memory=False)
    firstframe_points = df[df['FRAME'] == 0][['POSITION_X', 'POSITION_Y']].values
    center = np.mean(firstframe_points, axis=0)
    max_frame = df['FRAME'].max()
    max_radial_velocities = []
    
    trackids = df['TRACK_ID'].unique()
    for trackid in trackids:
        points_framesorted = df[df['TRACK_ID'] == trackid][['POSITION_X', 'POSITION_Y', 'FRAME']].sort_values(by=['FRAME'])
        frames = points_framesorted['FRAME'].values
        points_framesorted = points_framesorted[['POSITION_X', 'POSITION_Y']].values
        radial_vector = points_framesorted - center
        rhos = np.linalg.norm(radial_vector, axis=1)
        t0 = frames[0]
        rho0 = rhos[0]
        max_rho, max_t = max_upto_each_index(rhos)
        # skip first value since division by 0
        max_radial_velocity = [0 if t==0 else (r-rho0)/(t) for r,t in zip(max_rho[1:], max_t[1:])]
        # fill the first entries with nans so that in the end you can sum non nans, 
        # because some radial velocities can actually be 0s
        full_size_max_rad_vel = np.full((max_frame+1,), np.nan)
        full_size_max_rad_vel[t0+1:t0+1+len(max_radial_velocity)] = np.array(max_radial_velocity)
        max_radial_velocities.append(full_size_max_rad_vel)
    max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
    # rad vel for first frame is 0
    max_radial_velocities[0] = 0
    return max_radial_velocities

all_radial_velocities = []

for file in csv_files:
    radial_velocity = compute_radial_velocity_of_trackmate_csvfile(os.path.join(CSV_ROOT, file))
    all_radial_velocities.append(radial_velocity)
all_radial_velocities = np.array(all_radial_velocities)
# divide everything by 10 because these numbers are um/1Frame and 1Frame=10min and we want um/min
all_radial_velocities = all_radial_velocities / 10

# replace 0 with nan so we can compute the mean and std without including zeros
# all_radial_velocities[all_radial_velocities == 0] = np.nan
mean_radial_velocity = np.nanmean(all_radial_velocities, axis=0)
std_radial_velocity = np.nanstd(all_radial_velocities, axis=0)

refdf['radial-velocity-mean(um/min)'] = mean_radial_velocity
refdf['radial-velocity-std(um/min)'] = std_radial_velocity

refdf

/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/1026467854.py:40: RuntimeWarning: Mean of empty slice
  max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/1026467854.py:40: RuntimeWarning: Mean of empty slice
  max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/1026467854.py:40: RuntimeWarning: Mean of empty slice
  max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/1026467854.py:40: RuntimeWarning: Mean of empty slice
  max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/1026467854.py:40: RuntimeWarning: Mean of empty slice
  max_radial_velocities = np.nanmean(max_radial_velocities, axis=0)
/var/folders/34/7vrvs7ts667_lxxqxdk42tlh0000gp/T/ipykernel_93946/

,t,inf-count-mean,inf-count-std,radius-mean(um),radius-std(um),area-mean(um2),area-std(um2),radial-velocity-mean(um/min),radial-velocity-std(um/min)
0,20.00,8.454545,4.390071,50.775458,16.695897,6329.897458,5058.395235,0.000000,0.000000
1,20.17,8.818182,4.190899,51.469313,16.068189,6738.964437,4956.375937,0.070642,0.049695
2,20.33,9.363636,4.433345,52.236234,16.578712,6949.618190,5200.132492,0.069266,0.033415
3,20.50,10.272727,4.692354,55.131957,16.693659,7757.873227,5601.432696,0.064372,0.024805
4,20.67,10.636364,4.822297,57.225456,15.660132,8319.521243,5540.309176,0.066776,0.030413
...,...,...,...,...,...,...,...,...,...
164,47.33,540.545455,103.941679,502.870874,114.122022,818956.247324,256073.083318,0.057487,0.002722
165,47.50,546.545455,106.215219,493.045570,105.914216,827120.347954,253250.178784,0.057716,0.002784
166,47.67,551.454545,105.298019,500.333248,114.702915,835214.300623,250179.149789,0.057566,0.003046
167,47.83,557.818182,103.831419,502.874965,114.966366,836713.306692,246823.017673,0.056964,0.002588


## Part 6: Saving the reference metrics

In [40]:
save_path = os.path.join('..', 'output', 'reference_metrics_for_'+ dataset_name + '.csv')
refdf.to_csv(save_path, index=False)